<a href="https://colab.research.google.com/github/Nishijujuba/python-cookbook-2023-3rd/blob/master/_downloads/af0caf6d7af0dda755f4c9d7af9ccc2c/quickstart_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\| **Quickstart** \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Quickstart
==========

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

Working with data
-----------------

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.

Dataset 负责“样本+标签”的存取定义；DataLoader 把 Dataset 包装成可迭代对象（按 batch 取数据、可打乱、并行加载等）。


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
#导入 ToTensor：把 PIL 图片/数组转换为 PyTorch 张量，并按 TorchVision 约定缩放到 [0,1]（常见行为）。

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like CIFAR, COCO ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset`
includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.

transform：作用在输入样本（图像）上
target_transform：作用在标签上


In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(#创建训练集对象（Dataset）
    root="data",#数据保存目录为 data/
    train=True,#选择训练集划分
    download=True,#若本地没有就下载
    transform=ToTensor(),#读到的图片会经过 ToTensor() 转成张量
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(#创建测试集对象
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 212kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.95MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.36MB/s]


We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports automatic batching, sampling,
shuffling and multiprocess data loading. Here we define a batch size of
64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).


------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [10]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()#定义展平层：把输入从 (N, C, H, W) 变为 (N, C*H*W)（对单张也类似）。
        self.linear_relu_stack = nn.Sequential(#用 Sequential 串联多层
            nn.Linear(28*28, 512),#全连接：输入 784，输出 512。
            nn.ReLU(),#ReLU 激活
            nn.Linear(512, 512),#第二个全连接：512→512
            nn.ReLU(),#ReLU 激活
            nn.Linear(512, 10)#输出层：512→10，对应 10 类的 logits（未做 softmax）
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)#实例化模型，并移动到指定 device。
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](buildmodel_tutorial.html).


------------------------------------------------------------------------


Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [8]:
loss_fn = nn.CrossEntropyLoss()#多分类交叉熵损失（输入 logits + 目标类别索引）
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)#用 SGD 优化器更新模型参数，学习率 1×10−3


In a single training loop, the model makes predictions on the training
dataset (fed to it in batches), and backpropagates the prediction error
to adjust the model\'s parameters.


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()#切换到训练模式（启用 dropout/bn 的训练行为；本模型虽未用这些层，但这是规范写法）
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:#每 100 个 batch 打印一次日志。
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.


In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)#测试集样本数。
    num_batches = len(dataloader)#测试集 batch 数。
    model.eval()#切换到评估模式（关闭 dropout、bn 用滑动统计等）。
    test_loss, correct = 0, 0#累加测试损失与正确数。
    with torch.no_grad():#评估时不记录梯度，省内存更快。
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            #pred = model(X) 的输出是 logits，形状通常是 (batch_size, num_classes)，例如 (64, 10)。
            #pred[i, j] 表示第 i 个样本在第 j 类上的分数（未做 softmax 也没关系）。
            #做分类时，取分数最大的那个类就是预测类别。

            test_loss += loss_fn(pred, y).item()#累加 batch 损失。

            correct += (pred.argmax(1) == y).type(torch.float).sum().item()#得到每个样本预测类别；与 y 比较后统计正确个数并累加
            #argmax(1) 表示沿着维度 1（类别维）找最大值的索引。
            #结果形状是 (batch_size,)，每个元素是 0~9 之间的类别编号

            #y 是真实标签，形状也是 (batch_size,)，每个元素是类别编号。
            #比较后得到一个布尔张量 (batch_size,)：
            #True 表示该样本预测正确
            #False 表示预测错误

            #.type(torch.float)：把 True/False 转成 1/0
            #True → 1.0 False → 0.0
            #这样才能求和统计“正确个数”。

            #.item()取出 Python 数值，并累加到 correct



    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.


In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310149  [   64/60000]
loss: 2.306224  [ 6464/60000]
loss: 2.300238  [12864/60000]
loss: 2.293448  [19264/60000]
loss: 2.315622  [25664/60000]
loss: 2.319275  [32064/60000]
loss: 2.305428  [38464/60000]
loss: 2.316592  [44864/60000]
loss: 2.306747  [51264/60000]
loss: 2.302805  [57664/60000]
Test Error: 
 Accuracy: 10.3%, Avg loss: 2.307959 

Epoch 2
-------------------------------
loss: 2.310149  [   64/60000]
loss: 2.306224  [ 6464/60000]
loss: 2.300238  [12864/60000]
loss: 2.293448  [19264/60000]
loss: 2.315622  [25664/60000]
loss: 2.319275  [32064/60000]
loss: 2.305428  [38464/60000]
loss: 2.316592  [44864/60000]
loss: 2.306747  [51264/60000]
loss: 2.302805  [57664/60000]
Test Error: 
 Accuracy: 10.3%, Avg loss: 2.307959 

Epoch 3
-------------------------------
loss: 2.310149  [   64/60000]
loss: 2.306224  [ 6464/60000]
loss: 2.300238  [12864/60000]
loss: 2.293448  [19264/60000]
loss: 2.315622  [25664/60000]
loss: 2.319275  [32064/600

Read more about [Training your model](optimization_tutorial.html).


------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [13]:
torch.save(model.state_dict(), "model.pth")#把模型参数保存到 model.pth
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))
#从文件读取参数并加载到模型中；weights_only=True 表示只加载权重相关内容（较新接口的安全/兼容选项）

<All keys matched successfully>

This model can now be used to make predictions.


In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
#取测试集第 0 个样本：x 是图片张量，y 是真实标签索引。
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    #从 logits 里取最大值对应类别，映射到类别名；同时把真实标签映射成类别名。
    #pred[0]：取出（可能存在的）第一个维度对应的向量

    #model(x) 的输出通常叫 logits：形状一般是
    #单张图片（带 batch 维）：pred.shape == (1, 10)
    #一批图片：pred.shape == (N, 10)
    #pred = model(X) 的输出是 logits，形状通常是 (batch_size, num_classes)，例如 (64, 10)。
    #pred[i, j] 表示第 i 个样本在第 j 类上的分数（未做 softmax 也没关系）。
    #做分类时，取分数最大的那个类就是预测类别。

    #argmax(0)：取最大 logit 的索引


    #argmax 是 Tensor 的方法：返回“最大值所在的位置（索引）”。
    #常用写法：pred.argmax(dim=...)
    #返回的不是最大值本身，而是最大值的下标。

    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Bag", Actual: "Ankle boot"


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).
